In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings # Supress warnings 
warnings.filterwarnings('ignore')

np.random.seed(0)

In [ ]:
file_path="/kaggle/input/public-telecom-customer-churn-analysis-and-prediction/train_data.csv"
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()

# Check missing value

In [ ]:
data.isnull().sum()

In [ ]:
# Thay thế missing values trong các cột với phương pháp phù hợp
# Thay thế missing values của NetworkScore bằng giá trị trung bình
data['NetworkScore'].fillna(data['NetworkScore'].mean(), inplace=True)

# Thay thế missing values của Age bằng giá trị trung bình
data['Age'].fillna(data['Age'].mean(), inplace=True)

# Thay thế missing values của IsActiveMember bằng giá trị mode (giá trị xuất hiện nhiều nhất)
data['IsActiveMember'].fillna(data['IsActiveMember'].mode()[0], inplace=True)

# Thay thế missing values của EstimatedMonthlyUsage bằng giá trị trung bình
data['EstimatedMonthlyUsage'].fillna(data['EstimatedMonthlyUsage'].mean(), inplace=True)

# Kiểm tra lại xem đã xử lý hết missing values chưa
print(data.isnull().sum())

In [ ]:
data.describe()

# LLAMA

In [ ]:
!pip install together

In [ ]:
import pandas as pd
# import google.generativeai as genai # Không cần thiết nữa
import time
import json
import os
import re
from together import Together # Import thư viện Together AI

In [ ]:
# Thay thế bằng API key của bạn
API_KEY = ""  # Thay bằng API key thực tế

client = Together(api_key=API_KEY)

# Tên mô hình bạn muốn sử dụng trên Together AI
TOGETHER_MODEL = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"

In [ ]:
# test
response = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    messages=[
      {
        "role": "user",
        "content": "What are some fun things to do in New York?"
      }
    ]
)
print(response.choices[0].message.content)

# LLM

In [ ]:
SYSTEM_PROMPT = """
You are a predictive model tasked with determining whether a customer will churn (Exited: 1) or stay (Exited: 0) based on their telecom service data. Below are examples of customer data with their churn status:

Example 1:
CustomerID: 2858
Surname: Bell
NetworkScore: 68.0
Region: North
Gender: Female
Age: 60.0
Tenure: 17
MonthlyCharge: 25.72
NumOfProducts: 3
HasInternetService: 1
IsActiveMember: 1.0
EstimatedMonthlyUsage: 533.4
Exited: 0

Example 2:
CustomerID: 5794
Surname: Baker
NetworkScore: 34.0
Region: North
Gender: Male
Age: 28.0
Tenure: 7
MonthlyCharge: 131.03
NumOfProducts: 2
HasInternetService: 0
IsActiveMember: 0.0
EstimatedMonthlyUsage: 509.04
Exited: 0

Example 3:
CustomerID: 3996
Surname: Jackson
NetworkScore: 13.0
Region: South
Gender: Male
Age: 65.0
Tenure: 2
MonthlyCharge: 64.27
NumOfProducts: 3
HasInternetService: 0
IsActiveMember: 1.0
EstimatedMonthlyUsage: 246.48
Exited: 0

Example 4:
CustomerID: 2680
Surname: Craig
NetworkScore: 50.0
Region: East
Gender: Female
Age: 60.0
Tenure: 16
MonthlyCharge: 41.26
NumOfProducts: 3
HasInternetService: 1
IsActiveMember: 1.0
EstimatedMonthlyUsage: 902.28
Exited: 1

Example 5:
CustomerID: 1528
Surname: Drake
NetworkScore: 18.0
Region: North
Gender: Female
Age: 63.0
Tenure: 5
MonthlyCharge: 99.55
NumOfProducts: 4
HasInternetService: 0
IsActiveMember: 0.0
EstimatedMonthlyUsage: 949.95
Exited: 0

Example 6:
CustomerID: 217
Surname: Chambers
NetworkScore: 48.0
Region: South
Gender: Female
Age: 29.0
Tenure: 15
MonthlyCharge: 46.91
NumOfProducts: 1
HasInternetService: 1
IsActiveMember: 0.0
EstimatedMonthlyUsage: 847.45
Exited: 1

Example 7:
CustomerID: 2987
Surname: Salazar
NetworkScore: 98.0
Region: South
Gender: Male
Age: 60.0
Tenure: 20
MonthlyCharge: 193.17
NumOfProducts: 1
HasInternetService: 1
IsActiveMember: 1.0
EstimatedMonthlyUsage: 328.63
Exited: 0

Example 8:
CustomerID: 1414
Surname: Knight
NetworkScore: 64.0
Region: South
Gender: Other
Age: 53.0
Tenure: 4
MonthlyCharge: 145.4
NumOfProducts: 4
HasInternetService: 1
IsActiveMember: 1.0
EstimatedMonthlyUsage: 506.3
Exited: 0

Example 9:
CustomerID: 1113
Surname: Joseph
NetworkScore: 6.0
Region: North
Gender: Male
Age: 36.0
Tenure: 10
MonthlyCharge: 193.33
NumOfProducts: 2
HasInternetService: 1
IsActiveMember: 1.0
EstimatedMonthlyUsage: 862.39
Exited: 1
"""

In [ ]:
def create_user_prompt(test_sample):
    """
    Tạo phần user prompt chứa dữ liệu khách hàng cụ thể cần dự đoán.
    """
    user_prompt_text = f"""
Now, predict the Exited label (0 or 1) for the following customer:
CustomerID: {test_sample['CustomerID']}
Surname: {test_sample['Surname']}
NetworkScore: {test_sample['NetworkScore']}
Region: {test_sample['Region']}
Gender: {test_sample['Gender']}
Age: {test_sample['Age']}
Tenure: {test_sample['Tenure']}
MonthlyCharge: {test_sample['MonthlyCharge']}
NumOfProducts: {test_sample['NumOfProducts']}
HasInternetService: {test_sample['HasInternetService']}
IsActiveMember: {test_sample['IsActiveMember']}
EstimatedMonthlyUsage: {test_sample['EstimatedMonthlyUsage']}

Return ONLY the digit 0 or 1, representing the Exited status. Do not include any other text or formatting.
"""
    return user_prompt_text


In [ ]:
def predict_churn(test_sample):
    """
    Dự đoán nhãn Exited cho một mẫu test sử dụng Together AI API.
    Chỉ trả về số nguyên 0 hoặc 1.
    Sử dụng cả system prompt và user prompt.
    """
    try:
        # Tạo user prompt từ mẫu test
        user_prompt = create_user_prompt(test_sample)
        
        # Gửi yêu cầu tới Together AI API với cả system và user messages
        response = client.chat.completions.create(
            model=TOGETHER_MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT}, # System prompt
                {"role": "user", "content": user_prompt}     # User prompt
            ],
            # Bạn có thể thêm các tham số khác ở đây như temperature, max_tokens, v.v.
            # Ví dụ: temperature=0.0, max_tokens=1 (để khuyến khích chỉ trả về 1 ký tự)
            temperature=0.0, # Giảm tính ngẫu nhiên để mô hình tuân thủ định dạng chặt chẽ hơn
            max_tokens=1     # Giới hạn phản hồi chỉ là 1 token (0 hoặc 1)
        )
        
        # Lấy văn bản phản hồi thô và loại bỏ khoảng trắng thừa
        result_text = response.choices[0].message.content.strip()
        
        # Thêm độ trễ để tuân thủ giới hạn tần suất (rate limits) của Together AI (60 RPM)
        time.sleep(1) 
        
        try:
            # Cố gắng chuyển đổi trực tiếp phản hồi thành số nguyên
            prediction = int(result_text)
            # Kiểm tra xem giá trị có phải là 0 hoặc 1 không
            if prediction in [0, 1]:
                return prediction
            else:
                print(f"Giá trị dự đoán không hợp lệ cho CustomerID {test_sample['CustomerID']}: Expected 0 or 1, got {prediction}")
                return -1
        except ValueError:
            # Xử lý lỗi nếu phản hồi không phải là một số nguyên hợp lệ
            print(f"Lỗi chuyển đổi phản hồi thành số nguyên cho CustomerID {test_sample['CustomerID']}: '{result_text}' không phải là số nguyên hợp lệ.")
            return -1

    except Exception as e:
        # Bắt các lỗi chung khác trong quá trình gọi API hoặc xử lý
        print(f"Lỗi khi dự đoán cho CustomerID {test_sample['CustomerID']}: {e}")
        
        # Thêm độ trễ khi xảy ra lỗi để tránh gửi yêu cầu quá nhanh sau lỗi
        time.sleep(5) 
        return -1


In [ ]:
# Đọc dữ liệu test từ file CSV
test_data = pd.read_csv('/kaggle/input/public-telecom-customer-churn-analysis-and-prediction/test_data.csv')

In [ ]:
# Dự đoán cho từng mẫu test
y_test_pred = []
for index, test_sample in test_data.iterrows():
    prediction = predict_churn(test_sample)
    y_test_pred.append(prediction)
    print(f"Predicted Exited for CustomerID {test_sample['CustomerID']}: {prediction}")
    # Thêm khoảng nghỉ để tránh vượt giới hạn API
    time.sleep(1)

# Submission

In [ ]:
# Tạo DataFrame kết quả
result_df = pd.DataFrame({
    'CustomerID': test_data['CustomerID'],
    'Exited': y_test_pred
})

# Lưu kết quả vào file CSV
result_df.to_csv('churn_predictions.csv', index=False)
print("Predictions saved to churn_predictions.csv")

# Kiểm tra kết quả
print(result_df.head())